## Sample Project: Build a portfolio using Markowitz's modern portfolio theory

## Language: Python
## Asset: Technology Stocks in the S&P 500
## Notebook: Jupyter or Google Colab


### IMPORTING LIBRARIES

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
from scipy.optimize import minimize

##### technology Stock - Google owner alphabet,Amazon,Micrsoft,Apple

In [24]:
# Define the list of technology stocks in your portfolio
tech_stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN']

In [7]:
# Set the start and end date for historical data
start_date = '2022-01-01'
end_date = '2023-06-30'


##### obtaining historical stock price data for the technology stock in the S&P 500......... using APIs like YAHOO FINANCE Or Libraries like YFINANCE to fetch the Data

In [26]:
# Fetch historical data for the selected stocks
data = yf.download(tech_stocks, start=start_date, end=end_date)['Adj Close']


[*********************100%%**********************]  4 of 4 completed


In [9]:
data

,AAPL,AMZN,GOOGL,MSFT
Date,,,,
2022-01-03,180.190964,170.404495,144.991501,329.394867
2022-01-04,177.904053,167.522003,144.399506,323.746704
2022-01-05,173.171814,164.356995,137.774994,311.318756
2022-01-06,170.281021,163.253998,137.747498,308.858765
2022-01-07,170.449295,162.554001,137.016998,309.016205
...,...,...,...,...
2023-06-23,186.428238,129.330002,122.339996,334.312164
2023-06-26,185.020157,127.330002,118.339996,327.905762
2023-06-27,187.806381,129.179993,118.330002,333.863159


In [10]:
data.isnull().sum()

AAPL     0
AMZN     0
GOOGL    0
MSFT     0
dtype: int64

In [12]:
# Calculate daily returns
returns = data.pct_change().dropna()

In [13]:
returns

,AAPL,AMZN,GOOGL,MSFT
Date,,,,
2022-01-04,-0.012692,-0.016916,-0.004083,-0.017147
2022-01-05,-0.026600,-0.018893,-0.045876,-0.038388
2022-01-06,-0.016693,-0.006711,-0.000200,-0.007902
2022-01-07,0.000988,-0.004288,-0.005303,0.000510
2022-01-10,0.000116,-0.006570,0.012061,0.000732
...,...,...,...,...
2023-06-23,-0.001711,-0.006300,-0.006577,-0.013806
2023-06-26,-0.007553,-0.015464,-0.032696,-0.019163
2023-06-27,0.015059,0.014529,-0.000084,0.018168


In [14]:
# Calculate mean returns and covariance matrix
mean_returns = returns.mean()
cov_matrix = returns.cov()


In [15]:
# Define the objective function for portfolio optimization
def objective(weights):
    port_returns = np.sum(mean_returns * weights)
    port_stddev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    return -port_returns / port_stddev  # Negative Sharpe ratio for maximization

In [16]:
# Set optimization constraints (e.g., sum of weights equals 1)
constraints = ({'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1})

In [17]:
# Set bounds for asset weights (0 to 1)
bounds = tuple((0, 1) for _ in range(len(tech_stocks)))

In [20]:
# Initial guess for asset weights
initial_weights = len(tech_stocks) * [1.0 / len(tech_stocks)]


In [21]:
# Perform portfolio optimization to find optimal weights
result = minimize(objective, initial_weights, method='SLSQP', bounds=bounds, constraints=constraints)

In [22]:
# Print the optimal portfolio allocation
optimal_weights = result.x
print("Optimal Portfolio Allocation:")
for stock, weight in zip(tech_stocks, optimal_weights):
    print(f"{stock}: {weight:.4f}")

Optimal Portfolio Allocation:
AAPL: 1.0000
MSFT: 0.0000
GOOGL: 0.0000
AMZN: 0.0000


In [23]:
# Calculate and print the expected portfolio return and risk
expected_return = np.sum(mean_returns * optimal_weights)
portfolio_stddev = np.sqrt(np.dot(optimal_weights.T, np.dot(cov_matrix, optimal_weights)))
print(f"Expected Portfolio Return: {expected_return:.4f}")
print(f"Portfolio Standard Deviation: {portfolio_stddev:.4f}")

Expected Portfolio Return: 0.0003
Portfolio Standard Deviation: 0.0200
